### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.53' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.53'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.53 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.5.12 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.1.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.53


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.53'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.53'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.53'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.53'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  321 Jul 29 14:25 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1824 Jul 29 14:25 dockerfile_aml-sdk_docker_image_sdk.v1.0.53


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.53 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.53 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
4.6.14: Pulling from continuumio/miniconda3

b959ec00: Pulling fs layer 
08a50ae3: Pulling fs layer 
dbb53a8b: Pulling fs layer 
448383b9: Pulling fs layer 
Digest: sha256:97c73cf6b1aef82bb9f79d9ee82bdcb7bcef07d508fa270bc608bf61d7e98becExtracting  60.72MB/103.7MB
Status: Downloaded newer image for continuumio/miniconda3:4.6.14
 ---> 6b5cf97566c3
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in bc6415c8dc68
Removing intermediate container bc6415c8dc68
 ---> 98d3cbe9c5d4
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 4ff5b6409cfd
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRe

Get:42 http://deb.debian.org/debian stretch/main amd64 gcc amd64 4:6.3.0-4 [5196 B]
Get:43 http://deb.debian.org/debian stretch/main amd64 libc-dev-bin amd64 2.24-11+deb9u4 [259 kB]
Get:44 http://deb.debian.org/debian stretch/main amd64 libc6-dev amd64 2.24-11+deb9u4 [2364 kB]
Get:45 http://deb.debian.org/debian stretch/main amd64 libstdc++-6-dev amd64 6.3.0-18+deb9u1 [1420 kB]
Get:46 http://deb.debian.org/debian stretch/main amd64 g++-6 amd64 6.3.0-18+deb9u1 [7094 kB]
Get:47 http://deb.debian.org/debian stretch/main amd64 g++ amd64 4:6.3.0-4 [1546 B]
Get:48 http://deb.debian.org/debian stretch/main amd64 lsb-release all 9.20161125 [27.1 kB]
Get:49 http://deb.debian.org/debian stretch/main amd64 make amd64 4.1-9.1 [302 kB]
Get:50 http://deb.debian.org/debian stretch/main amd64 sudo amd64 1.8.19p1-2.1 [1055 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 43.6 MB in 0s (94.6 MB/s)
Selecting previously unselected package libpython3.5-minimal:amd64.
(R

Selecting previously unselected package libc6-dev:amd64.
Preparing to unpack .../31-libc6-dev_2.24-11+deb9u4_amd64.deb ...
Unpacking libc6-dev:amd64 (2.24-11+deb9u4) ...
Selecting previously unselected package libstdc++-6-dev:amd64.
Preparing to unpack .../32-libstdc++-6-dev_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libstdc++-6-dev:amd64 (6.3.0-18+deb9u1) ...
Selecting previously unselected package g++-6.
Preparing to unpack .../33-g++-6_6.3.0-18+deb9u1_amd64.deb ...
Unpacking g++-6 (6.3.0-18+deb9u1) ...
Selecting previously unselected package g++.
Preparing to unpack .../34-g++_4%3a6.3.0-4_amd64.deb ...
Unpacking g++ (4:6.3.0-4) ...
Selecting previously unselected package lsb-release.
Preparing to unpack .../35-lsb-release_9.20161125_all.deb ...
Unpacking lsb-release (9.20161125) ...
Selecting previously unselected package make.
Preparing to unpack .../36-make_4.1-9.1_amd64.deb ...
Unpacking make (4.1-9.1) ...
Selecting previously unselected package sudo.
Preparing to unpack .../37-sudo

Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
defusedxml-0.6.0-py_0.tar.bz2                 23 KB
markupsafe-1.1.1-py36h7b6447c_0.tar.bz2       29 KB
zlib-1.2.11-h7b6447c_3.tar.bz2               120 KB
blas-1.0-mkl.tar.bz2                           6 KB
ca-certificates-2019.5.15-0.tar.bz2          133 KB
bleach-3.1.0-py36_0.tar.bz2                  226 KB
testpath-0.4.2-py36_0.tar.bz2                 91 KB
pip-19.1.1-py36_0.tar.bz2                    1.9 MB
nb_conda-2.2.1-py36_0.tar.bz2                 33 KB
jedi-0.13.3-py36_0.tar.bz2                   236 KB
mkl-2019.4-243.tar.bz2                     204.1 MB
ptyprocess-0.6.0-py36_0.tar.bz2               23 KB
entrypoints-0.3-py36_0.tar.bz2                12 KB
readline-7.0-h7b6447c_5.tar.bz2              392 KB
wheel-0.33.4-py36_0.tar.bz2                   40 KB
jinja2-2.10.1-py36_0.tar.bz2                 188 KB
mistune-0.8.4-py36h7b6447c_0.tar.bz2          54 KB
parso-0.5.0-py_

 ---> Running in 8a7234f7cb05
Removing intermediate container 8a7234f7cb05
 ---> 6ae4ed3855f7
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 5e90bb0c3575
Removing intermediate container 5e90bb0c3575
 ---> 9c7d8c8cb854
Successfully built 9c7d8c8cb854
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.53
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly
 ---> 6b5cf97566c3
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> 98d3cbe9c5d4
Step 3/17 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 7cd7163f276f
Step 4/17 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> Using cache
 ---> 5d6fe7f9f788
Step 5/17 : ENV CONDA

  5750K .......... .......... .......... .......... .......... 70% 83.0M 0s
  5800K .......... .......... .......... .......... .......... 70%  163M 0s
  5850K .......... .......... .......... .......... .......... 71%  117M 0s
  5900K .......... .......... .......... .......... .......... 71% 99.9M 0s
  5950K .......... .......... .......... .......... .......... 72% 87.3M 0s
  6000K .......... .......... .......... .......... .......... 73%  140M 0s
  6050K .......... .......... .......... .......... .......... 73%  101M 0s
  6100K .......... .......... .......... .......... .......... 74%  126M 0s
  6150K .......... .......... .......... .......... .......... 74%  101M 0s
  6200K .......... .......... .......... .......... .......... 75% 81.5M 0s
  6250K .......... .......... .......... .......... .......... 76% 90.8M 0s
  6300K .......... .......... .......... .......... .......... 76%  111M 0s
  6350K .......... .......... .......... .......... .......... 77% 75.6M 0s
  6400K ....

 ---> Running in 089bc3b98362
Removing intermediate container 089bc3b98362
 ---> 6fe2ed448898
Step 20/20 : CMD /bin/bash
 ---> Running in 82640e6a4362
Removing intermediate container 82640e6a4362
 ---> 4d49c19afaef
Successfully built 4d49c19afaef
Successfully tagged georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.53


In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 8
-rwxrwxrwx 1 root root 5030 Jun 27 18:19 exported_conda_env.yml


In [11]:
# export the resulted conda env
!docker run  \
        -v $crt_local_dir:/workspace:rw \
        $aml_docker_image_name \
        /bin/bash -c "conda env export > /workspace/exported_conda_env.yml"

In [12]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [13]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.53 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [14]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.53 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.53
1.0.53
azure-cli                         2.0.69

command-modules-nspkg               2.0.3
core                              2.0.69
nspkg                              3.0.4
telemetry                          1.0.3

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Jul 11 2019, 08:40:07) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.
/bin/bash: azcopy: command not found
1.0.53
azure-cli                         2.0.69

command-modules-nspkg               2.0.3
core                              2.0.69
nspkg                              3.0.4
telemetry                          1.0.3

Extensions:
azure-devops                      0.11.0

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Pytho

#### Do the same in a  notebook

In [15]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [16]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [17]:
!ls -l $crt_local_dir

total 12
-rwxrwxrwx 1 root       root       4917 Jul 29 14:30 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001  982 Jul 29 14:30 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [18]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [19]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.53\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [20]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

33af90ae: Preparing 
f5841ce2: Preparing 
adf75e8e: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 


33af90ae: Pushing  1.874GB/2.638GBPushing    300MB/2.638GBPushing  325.6MB/2.638GB

33af90ae: Pushed   2.691GB/2.638GBPushing  2.027GB/2.638GBsdk.v1.0.53: digest: sha256:8f4fedb1a7a43e96232b24dfaed5f3328078b72e90361bafef8575ea20bc4ded size: 2003
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

21bdbe23: Preparing 
5d2143a9: Preparing 
30ea948b: Preparing 
33af90ae: Preparing 
f5841ce2: Preparing 
adf75e8e: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
30ea948b: Pushed   465.2MB/431.9MBraccount/aml-sdk_docker_image sdk.v1.0.53: digest: sha256:fde2f562875bf5d6aedfa9b7d015d93691dbcdc9e97f7e109a9383218e999790 size: 2635
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

2ffd8000: Preparing 
21bdbe23: Preparing 
5d2143a9: Preparing 
30ea948b: Preparing 
33af90ae: Preparing 
f5841ce2: Preparing 
adf75e8e: Preparing 
c65c8dc4: Preparing 
2f5d7ee9: Preparing 
b1358a5a: Preparing 
1ff9ade6: Preparing 
ffd8000: Pushed   16.98MBists MBeraccount/a

In [21]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 506398 bytes to createAMLSDKDocker.html
